In [1]:
import os

### Функция прогнозирования

In [11]:
def forecast(year):

    models_dir = f'results\Models\{year}'

    # Получить список файлов .ipnb из results\Models\<year>
    file_list = tuple(filter(lambda fn: '.pickle' in fn, os.listdir(models_dir)))
    print(file_list)

    # Сделать множество из кортежей (Название-датасета, группа), преобразовать в список, отсортировать
    dataset_group = set()
    for fn in file_list:
        dataset, year, group, model = fn.split('_')
        dataset_group |= {(dataset, group.split('гр')[1])}
    ds_list = sorted(dataset_group)
    print(ds_list)

    # Для каждого элемента списка создать result_list[one_model_raw: dict]
    
    for ds in ds_list:
        result_list = []
        ds_models = filter(lambda fn: ds[0] in fn and ds[1] in fn, file_list)
        for file_name in ds_models:
            with open(f'results/models/{year}', 'rb') as f:
                one_model_row = pickle.load(f)
                model = model_info['Model_full']
                y_predicted = np.ravel(model.predict(X_new))
                one_model_row['Prediction'] = y_predicted
                result_list.append(one_model_row)
                # Сортировка результатов по каждому датасету
                result_list.sort(key=lambda row: (row['Criterion'], -row['Correlation'], -row['Pm']))
    
    

In [12]:
forecast(2024)

('Вилия-Стешицы_2024_гр0_ARDRegression.pickle', 'Вилия-Стешицы_2024_гр0_BayesianRidge.pickle', 'Вилия-Стешицы_2024_гр0_BayesianRidgeCV.pickle', 'Вилия-Стешицы_2024_гр0_ElasticNetCV.pickle', 'Вилия-Стешицы_2024_гр0_LinearRegression.pickle', 'Вилия-Стешицы_2024_гр0_Ridge.pickle', 'Вилия-Стешицы_2024_гр0_RidgeCV.pickle', 'Вилия-Стешицы_2024_гр1_ARDRegression.pickle', 'Вилия-Стешицы_2024_гр1_BayesianRidge.pickle', 'Вилия-Стешицы_2024_гр1_BayesianRidgeCV.pickle', 'Вилия-Стешицы_2024_гр1_ElasticNetCV.pickle', 'Вилия-Стешицы_2024_гр1_LinearRegression.pickle', 'Вилия-Стешицы_2024_гр1_Ridge.pickle', 'Вилия-Стешицы_2024_гр1_RidgeCV.pickle', 'Вилия-Стешицы_2024_гр2_ARDRegression.pickle', 'Вилия-Стешицы_2024_гр2_BayesianRidge.pickle', 'Вилия-Стешицы_2024_гр2_BayesianRidgeCV.pickle', 'Вилия-Стешицы_2024_гр2_ElasticNetCV.pickle', 'Вилия-Стешицы_2024_гр2_LinearRegression.pickle', 'Вилия-Стешицы_2024_гр2_Ridge.pickle', 'Вилия-Стешицы_2024_гр2_RidgeCV.pickle', 'Неман-Белица_2024_гр0_ARDRegression.pickl